In [ ]:
import copy
import numpy as np
from os.path import join as pj
from PIL import Image
import torch
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ia.seed(1)

In [ ]:
image = np.asarray(Image.open(pj("/Users/user/Desktop/git/Insect_Phenology_Detector/data/train_refined_images", "20180614-1959" + ".png")))
default_height, default_width, _ = image.shape

In [ ]:
lbl_to_name = {
    0: "Insect"
}

In [ ]:
with open(pj("/Users/user/Desktop/git/Insect_Phenology_Detector/data/train_detection_data/refinedet_all", "20180614-1959" + ".txt")) as f:
    target_lines = f.readlines()

bbs_list = []
for target_line in target_lines:
    target_line = target_line.split("\n")[0]
    target_list = target_line.split(" ")
    x1 = float(target_list[0]) * default_width
    x2 = float(target_list[2]) * default_width
    y1 = float(target_list[1]) * default_height
    y2 = float(target_list[3]) * default_height
    bbs_list.append(BoundingBox(x1 = x1, x2 = x2, y1 = y1, y2 = y2, label = lbl_to_name[int(target_list[4])]))

bbs = BoundingBoxesOnImage(bbs_list, shape=image.shape)

In [ ]:
def crop_random(image, resize_size, crop_num, bbs=None):
    """
        crop img, type == "RANDOM"
        Args:
            - img: np.array, shape == [1, height, width, channel]
            - resize_size: int, resized size of image
            - crop_num: (int, int), (height_crop_num, width_crop_num)
            but in this function, use height_crop_num * width_crop_num as number of crop image
            - bbs: imgaug BoundingBox
    """
    height_after_crop = int(image.shape[0] / crop_num[0])
    width_after_crop = int(image.shape[1] / crop_num[1])
    
    image_aug_list = []
    bbs_aug_list = []
    for i in range(crop_num[0] * crop_num[1]):
        # set augmentations
        aug_seq = iaa.Sequential([
            iaa.CropToFixedSize(width=width_after_crop, height=height_after_crop, position="uniform"),
            iaa.Resize({"width": resize_size, "height": resize_size})
        ])
        # augment img and target
        if bbs is not None:
            image_aug, bbs_aug = aug_seq(image=image, bounding_boxes=bbs)
            # check coord in img shape
            bbs_aug_before_check = bbs_aug.bounding_boxes
            bbs_aug_after_check = copy.copy(bbs_aug.bounding_boxes)
            for bb in bbs_aug_before_check:
                if bb.is_fully_within_image(image_aug.shape):
                    pass
                else:
                    bbs_aug_after_check.remove(bb)
            # append img and target
            if len(bbs_aug_after_check) > 0:
                image_aug_list.append(image_aug)
                bbs_aug_list.append(bbs_aug_after_check)
        else:
            image_aug = aug_seq(image=image)
    
    return np.array(image_aug_list), bbs_aug_list

In [ ]:
def crop_spread_all_over(image, resize_size, crop_num, bbs=None):
    """
        crop img, type == "SPREAD_ALL_OVER"
        Args:
            - img: np.array, shape == [1, height, width, channel]
            - resize_size: int, resized size of image
            - crop_num: (int, int), (height_crop_num, width_crop_num)
            - bbs: imgaug BoundingBox
    """
    height_after_crop = int(image.shape[0] / crop_num[0])
    width_after_crop = int(image.shape[1] / crop_num[1])
    
    height_mov_ratio_per_crop = 1.0 / (crop_num[0] - 1)
    width_mov_ratio_per_crop = 1.0 / (crop_num[1] - 1)
    
    image_aug_list = []
    bbs_aug_list = []
    # create crop img
    for i in range(crop_num[0]):
        for j in range(crop_num[1]):
            # set augmentations
            aug_seq = iaa.Sequential([
                iaa.CropToFixedSize(width=width_after_crop, height=height_after_crop, position=(width_mov_ratio_per_crop * j, height_mov_ratio_per_crop * i)),
                iaa.Resize({"width": resize_size, "height": resize_size})
            ])
            # augment img and target
            if bbs is not None:
                image_aug, bbs_aug = aug_seq(image=image, bounding_boxes=bbs)
                # check coord in img shape
                bbs_aug_before_check = bbs_aug.bounding_boxes
                bbs_aug_after_check = copy.copy(bbs_aug.bounding_boxes)
                for bb in bbs_aug_before_check:
                    if bb.is_fully_within_image(image_aug.shape):
                        pass
                    else:
                        bbs_aug_after_check.remove(bb)
                # append img and target
                if len(bbs_aug_after_check) > 0:
                    image_aug_list.append(image_aug)
                    bbs_aug_list.append(bbs_aug_after_check)
            else:
                image_aug = aug_seq(image=image)
    
    return np.array(image_aug_list), bbs_aug_list

In [ ]:
#image_crop, bbs_crop_list = crop_spread_all_over(image, 512, (5, 5), bbs)
image_crop, bbs_crop_list = crop_random(image, 512, (5, 5), bbs)
bbs_crop = [BoundingBoxesOnImage(bbs_crop_list[idx], shape=image_crop[idx].shape) for idx in range(len(bbs_crop_list))]

In [ ]:
image_crop.shape

In [ ]:
len(bbs_crop)

In [ ]:
idx = 0

In [ ]:
plt.figure(figsize=(10,10))
image_after = bbs_crop[idx].draw_on_image(image_crop[idx], size=2)
plt.imshow(image_after)
idx += 1

In [ ]:
method_aug = ["HorizontalFlip", "VerticalFlip", "Rotate"]

In [ ]:
def adopt_augmentation(image_crop, bbs_crop, method_aug=None):
    """
        adopt augmentation to image_crop, bbs_crop
        Args:
            - image_crop: np.array, shape == [crop_num, height, width, channels], cropped images
            - bbs_crop: [BoundingBoxesOnImage, ...], imgaug bounding box
            - method_aug: [str, ...], adopt augmentation list
    """
    if method_aug is None:
        return image_crop, bbs_crop
    else:
        aug_list = []
        # create augmentation
        for augmentation in method_aug:
            if augmentation == "HorizontalFlip":
                aug_list.append(iaa.Fliplr(0.5))
            elif augmentation == "VerticalFlip":
                aug_list.append(iaa.Flipud(0.5))
            elif augmentation == "Rotate":
                aug_list.append(iaa.Rotate((-45, 45)))
            else:
                print("not implemented!: insect_dataset_from_voc_style_txt.adopt_augmentation")
        
        aug_seq = iaa.SomeOf(1, aug_list)
        
        image_crop_aug = []
        bbs_crop_aug = []
        # adopt augmentation
        for im_crop, bb_crop in zip(image_crop, bbs_crop):
            im_crop_aug, bb_crop_aug = aug_seq(image=im_crop, bounding_boxes=bb_crop)
            # check coord in im_crop_aug shape
            bb_crop_aug_before_check = bb_crop_aug.bounding_boxes
            bb_crop_aug_after_check = copy.copy(bb_crop_aug.bounding_boxes)
            for bb in bb_crop_aug_before_check:
                if bb.is_fully_within_image(im_crop_aug.shape):
                    pass
                else:
                    bb_crop_aug_after_check.remove(bb)
            # append im_crop_aug and bb_crop_aug
            if len(bb_crop_aug_after_check) > 0:
                image_crop_aug.append(im_crop_aug)
                bbs_crop_aug.append(bb_crop_aug_after_check)
        
        return np.array(image_crop_aug), bbs_crop_aug

In [ ]:
image_crop_aug, bbs_crop_aug_list = adopt_augmentation(image_crop, bbs_crop, method_aug)
bbs_crop_aug = [BoundingBoxesOnImage(bbs_crop_aug_list[idx], shape=image_crop_aug[idx].shape) for idx in range(len(bbs_crop_aug_list))]

In [ ]:
image_crop_aug.shape

In [ ]:
len(bbs_crop_aug)

In [ ]:
idx = 0

In [ ]:
plt.figure(figsize=(10,10))
image_after = bbs_crop_aug[idx].draw_on_image(image_crop_aug[idx], size=2)
plt.imshow(image_after)
idx += 1

In [ ]:
name_to_lbl = {
    "Insect": 0
}

In [ ]:
def create_pytorch_annotation(bbs_crop_aug, resize_size, name_to_lbl):
    """
        extract bounding box and convert to pytorch style
        Args:
            - bbs_crop_aug: [BoundingBoxesOnImage, ...], imgaug bounding box
    """
    target = []
    for bb_crop_aug in bbs_crop_aug:
        target_per_image = []
        for elem_bb in bb_crop_aug:
            x1 = elem_bb.x1 / float(resize_size)
            y1 = elem_bb.y1 / float(resize_size)
            x2 = elem_bb.x2 / float(resize_size)
            y2 = elem_bb.y2 / float(resize_size)
            lbl = int(name_to_lbl[elem_bb.label])
            target_per_image.append([x1, y1, x2, y2, lbl])
        target.append(torch.Tensor(target_per_image))
    return target

In [ ]:
target = create_pytorch_annotation(bbs_crop_aug, 512, name_to_lbl)